<h1><center><font size="6">Transfer Learning with Keras for Cifar-10</font></center></h1>

# Introduction


## Dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.


# Load packages

In [1]:
# Setup one GPU for tensorflow
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# The GPU id to use, "0", "1", etc.
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

# https://keras.io/applications/#documentation-for-individual-models
from keras.applications.resnet50 import ResNet50
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import tensorflow as tf

# Limit tensorflow gpu usage
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
sess = tf.Session(config=config)


Using TensorFlow backend.


## Parameters

In [13]:
# Some constants
IMG_ROWS = 32
IMG_COLS = 32
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2018

N_TRAIN_SAMPLES = 5000
N_TEST_SAMPLES = 1000

#Model
NUM_EPOCHS = 10
BATCH_SIZE = 64
NET_IMG_ROWS = 224
NET_IMG_COLS = 224

# Load data

In [3]:
(x_train_data, y_train_data), (x_test_data, y_test_data) = cifar10.load_data()

# Create a dictionary for each type of label 
LABELS = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}


In [4]:
# For this tutorial we will work on a smaller subset
train_idxs = np.random.choice(np.arange(x_train_data.shape[0]), size=N_TRAIN_SAMPLES, replace=False)
test_idxs = np.random.choice(np.arange(x_test_data.shape[0]), size=N_TEST_SAMPLES, replace=False)

x_train_data = x_train_data[train_idxs, :, :, :]
y_train_data = y_train_data[train_idxs]
x_test_data = x_test_data[test_idxs, :, :, :]
y_test_data = y_test_data[test_idxs]


In [5]:
print("Cifar-10 x_train shape: {}".format(x_train_data.shape))
print("Cifar-10 y_train shape: {}".format(y_train_data.shape))
print("Cifar-10 x_test shape: {}".format(x_test_data.shape))
print("Cifar-10 y_test shape: {}".format(y_test_data.shape))

Cifar-10 x_train shape: (5000, 32, 32, 3)
Cifar-10 y_train shape: (5000, 1)
Cifar-10 x_test shape: (1000, 32, 32, 3)
Cifar-10 y_test shape: (1000, 1)


## Data preprocessing

In [6]:
def data_preprocessing(x_data, y_data):
    out_x = np.array([cv2.resize(img, (NET_IMG_COLS, NET_IMG_ROWS)).astype(np.float32)
                      for img in x_data])
    out_y = to_categorical(y_data, len(np.unique(y_data)))
    
    return out_x, out_y

# prepare the data
X, y = data_preprocessing(x_train_data, y_train_data)
X_test, y_test = data_preprocessing(x_test_data, y_test_data)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)


# Build model

In [7]:
# Create the base model with weights pre-trained on ImageNet.
# We will use the Resnet50 model
# - [Deep Residual Learning for Image Recognition](
#    https://arxiv.org/abs/1512.03385)
base_model = ResNet50(
    input_shape=(NET_IMG_ROWS, NET_IMG_COLS, 3),  # Input image size
    weights='imagenet',                           # Use imagenet pre-trained weights
    include_top=False,                            # Drop classification layer
    pooling='avg'                                 # Global AVG pooling for the 
)
#  output feature vector


94658560/94653016 [==============================] - 517s 5us/step


### Inspect the model

Let's check the model we initialized.

In [8]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

### Add the classification layer

In [9]:
# get the output feature vector from the base model
x = base_model.output
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


In [10]:
# Check the final model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

# Training

In [11]:
# Training data generator
datagen_train = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
)

# Validation data generator
datagen_val = ImageDataGenerator(rescale=1./255)


In [14]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NUM_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE
)


Epoch 1/10
62/62 [==============================] - 48s 770ms/step - loss: 2.4734 - acc: 0.1041 - val_loss: 2.4700 - val_acc: 0.0900
Epoch 2/10
62/62 [==============================] - 47s 753ms/step - loss: 2.4634 - acc: 0.1074 - val_loss: 2.4700 - val_acc: 0.0900
Epoch 3/10
62/62 [==============================] - 46s 749ms/step - loss: 2.4636 - acc: 0.1023 - val_loss: 2.4700 - val_acc: 0.0900
Epoch 4/10
62/62 [==============================] - 45s 732ms/step - loss: 2.4754 - acc: 0.1026 - val_loss: 2.4700 - val_acc: 0.0900
Epoch 5/10
62/62 [==============================] - 38s 616ms/step - loss: 2.4613 - acc: 0.1089 - val_loss: 2.4700 - val_acc: 0.0900
Epoch 6/10
62/62 [==============================] - 43s 694ms/step - loss: 2.4796 - acc: 0.1018 - val_loss: 2.4700 - val_acc: 0.0900
Epoch 7/10
62/62 [==============================] - 44s 713ms/step - loss: 2.4552 - acc: 0.1079 - val_loss: 2.4700 - val_acc: 0.0900
Epoch 8/10
62/62 [==============================] - 44s 710ms/step - 

In [15]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
    print(i, layer.name)


0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 activatio

In [16]:
# We will finetune from add_9
for layer in model.layers[:99]:
    layer.trainable = False
for layer in model.layers[99:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NUM_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE
)

Epoch 1/10
62/62 [==============================] - 48s 774ms/step - loss: 2.3058 - acc: 0.1467 - val_loss: 2.4846 - val_acc: 0.0960
Epoch 2/10
62/62 [==============================] - 47s 756ms/step - loss: 2.0818 - acc: 0.2508 - val_loss: 2.4754 - val_acc: 0.1000
Epoch 3/10
62/62 [==============================] - 47s 751ms/step - loss: 1.8872 - acc: 0.3841 - val_loss: 2.5097 - val_acc: 0.1000
Epoch 4/10
62/62 [==============================] - 46s 747ms/step - loss: 1.7302 - acc: 0.4504 - val_loss: 2.5552 - val_acc: 0.1000
Epoch 5/10
62/62 [==============================] - 46s 739ms/step - loss: 1.5972 - acc: 0.5078 - val_loss: 2.6059 - val_acc: 0.1120
Epoch 6/10
62/62 [==============================] - 45s 732ms/step - loss: 1.4655 - acc: 0.5630 - val_loss: 2.6256 - val_acc: 0.1000
Epoch 7/10
62/62 [==============================] - 45s 722ms/step - loss: 1.3640 - acc: 0.5890 - val_loss: 2.6429 - val_acc: 0.1110
Epoch 8/10
62/62 [==============================] - 44s 717ms/step - 